## Gate cutting to reduce circuit width

In [1]:
import numpy as np
from qiskit.circuit.library import EfficientSU2
from qiskit.quantum_info import PauliList

Specify a circuit and some observables

In [2]:
circuit = EfficientSU2(4, entanglement="linear", reps=2).decompose()
circuit.assign_parameters([0.4] * len(circuit.parameters), inplace=True)

circuit.draw("mpl", scale=0.8)

### Specify some observables

Currently, only `Pauli` observables with phase equal to 1 are supported. Full support for `SparsePauliOp` is expected in CKT v0.4.0.

In [3]:
observables = PauliList(["ZZII", "IZZI", "IIZZ", "XIXI", "ZIZZ", "IXIX"])

Cut the circuit and observables by using `partition_problem` and specifying qubit partitions

In [3]:
from circuit_knitting.cutting import partition_problem

partitioned_problem = partition_problem(
    circuit=circuit,
    num_samples=np.inf,
    observables=observables,
    partition_labels="AABB",
)
partitioned_problem._fields

('subcircuits', 'subexperiments', 'subobservables', 'weights', 'bases')

Run the subexperiments for each partition using the Qiskit Aer Sampler primitive.

In [4]:
from qiskit_aer.primitives import Sampler

# Set up Qiskit Aer Sampler primitives.
sampler_a = Sampler(run_options={"shots": 2**12})
sampler_b = Sampler(run_options={"shots": 2**12})

# Retrieve results from each subexperiment
results_a = sampler_a.run(partitioned_problem.subexperiments["A"]).result()
results_b = sampler_b.run(partitioned_problem.subexperiments["B"]).result()

To run using Qiskit Runtime Sampler primitive, replace the code above with this commented block.

In [5]:
# from qiskit_ibm_runtime import Session, Options, Sampler

# with Session(backend="ibmq_qasm_simulator") as session:
#    options = Options()
#    options.execution.shots = 2**14
#    sampler_a = Sampler(options=options)
#    sampler_b = Sampler(options=options)

#    job_a = sampler_a.run(partitioned_problem.subexperiments["A"])
#    job_b = sampler_b.run(partitioned_problem.subexperiments["B"])

#    results_a = job_a.result()
#    results_b = job_b.result()

#    session.close()

Reconstruct the full expectation value, given the results of cutting

In [6]:
from circuit_knitting.cutting import reconstruct_expectation_values

# Include the number of bits used for cutting measurements in the results
for i in range(len(partitioned_problem.subexperiments["A"])):
    results_a.metadata[i]["num_qpd_bits"] = len(
        partitioned_problem.subexperiments["A"][i].cregs[0]
    )
for i in range(len(partitioned_problem.subexperiments["B"])):
    results_b.metadata[i]["num_qpd_bits"] = len(
        partitioned_problem.subexperiments["B"][i].cregs[0]
    )

# Combine results from each partition into a single results dictionary
results = {"A": results_a, "B": results_b}

simulated_expvals = reconstruct_expectation_values(
    partitioned_problem.subobservables,
    partitioned_problem.weights,
    results,
)
simulated_expvals

[0.3848624825477599,
 0.510217249393463,
 0.5903419256210325,
 0.11537766456604004,
 0.31011974811553944,
 -0.12901473045349124]